## Use VGG16 for Transfer Learning on CIFAR10

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms
from torchvision import models #vgg16


#### Downloading and preparing the CIFAR10 dataset

In [10]:
transform = transforms.Compose([
    transforms.Resize((224, 224)) #VGG16 takes 224px sized imgs
    , transforms.ToTensor()
    , transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = torchvision.datasets.CIFAR10(root = './data/CIFAR10'
                                      , train = True
                                      , download = True
                                      , transform = transform)

trainset, validset = torch.utils.data.random_split(dataset, [40000, 10000])

trainloader = DataLoader(trainset
                        , batch_size = 32
                        , shuffle = True
                        , num_workers = 1)

validloader = DataLoader(validset
                        , batch_size = 32
                        , shuffle = True
                        , num_workers = 1)

testset = torchvision.datasets.CIFAR10(root = './data/CIFAR10'
                                      , train = False
                                      , download = True
                                      , transform = transform)

testloader = DataLoader(testset
                       , batch_size = 32
                       , shuffle = False
                       , num_workers = 1)

Files already downloaded and verified
Files already downloaded and verified


#### Download and Prepare VGG16 network

In [15]:
vgg16 = models.vgg16(pretrained = True)
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [17]:
#We can see that the last layer classifies 1000 classes - we only need 10
#Further, we freeze conv layers, as they learned from the imagenet dataset - this
# can be played with

vgg16.classifier[6].out_features = 10

for param in vgg16.features.parameters():
    param.requires_grad = False

#### Training and Validation Functions

In [18]:
def validate(model, val_dataloader):
    model.eval()
    val_running_loss = 0
    val_running_corr = 0
    for batch in val_dataloader:
        images, labels = batch
        preds = model(images)
        loss = criterion(preds, labels)
        
        val_running_loss += loss.item()
        val_running_corr += preds.argmax(dim = 1).eq(labels).sum().item()
        
    val_loss = val_running_loss / len(val_dataloader.dataset)
    val_accuracy = 100 * val_running_corr / len(val_dataloader.dataset)
    
    print(f'Valid Loss: {val_loss:.4f}, Valid Acc %: {val_accuracy:.2f}')
    
    return val_loss, val_accuracy

In [19]:
def train(model, train_dataloader):
    model.train()
    train_running_loss = 0
    train_running_corr = 0
    for batch in train_dataloader:
        images, labels = batch
        optimizer.zero_grad()
        preds = model(images)
        loss = criterion(preds, labels)
        
        train_running_loss += loss.item()
        train_running_corr += preds.argmax(dim = 1).eq(labels).sum().item()
        
        loss.backward()
        optimizer.step()
        
        
    train_loss = train_running_loss / len(train_dataloader.dataset)
    train_accuracy = 100 * train_running_corr / len(train_dataloader.dataset)
    
    print(f'Train Loss: {train_loss:.4f}, Train Acc %: {train_accuracy:.2f}')
    
    return train_loss, train_accuracy

#### Training Loop - not trainable on local machine

In [20]:
optimizer = optim.SGD(vgg16.classifier.parameters()
                     , lr = 0.01
                     , momentum = 0.9) #???

criterion = nn.CrossEntropyLoss()

In [ ]:
train_loss, train_accuracy = [], []
val_loss, val_accuracy = [], []

for epoch in range(10):
    print('Epoch ', epoch)
    train_epoch_loss, train_epoch_acc = train(vgg16, trainloader)
    val_epoch_loss, val_epoch_acc = validate(vgg16, validloader)
    
    train_loss.append(train_epoch_loss)
    train_accuracy.append(train_epoch_acc)
    val_loss.append(val_epoch_loss)
    val_accuracy.append(val_epoch_acc)

#### Test Loop - not possible on local machine

In [22]:
def test(model, test_loader):
    test_correct = 0
    
    with torch.no_grad():
        model.eval()
        for batch in test_loader:
            images, labels = batch
            preds = model(images)
        
            test_correct += preds.argmax(dim = 1).eq(labels).sum().item()
    
    test_accuracy = 100 * test_correct / len(test_loader.dataset)    
    print('Accuracy on TestSet in %:', test_accuracy)
    
    #return test_correct, test_accuracy

In [ ]:
test(vgg16, testloader)